# $(\mathrm{H}_2\mathrm{O})_2$ cost estimate

In [1]:
import numpy as np
import openfermion
import pyscf
from pyscf import gto

/home/mitarai/anaconda/envs/qulacs/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
xyz_file = "TIP4P-2.xyz"

Get energy gap of a single molecule

In [3]:
atom = [('O', [-0.3511375701099324, -2.220327851906733, 1.4480207843187745]), ('H', [-0.24292674995680064, -1.697798925312798, 3.1763661875721043]), ('H', [-0.11012983703262759, -0.699680546250837, 0.4985624750191387]),]
basis = 'sto3g'
mol = gto.M(atom=atom,
            basis=basis,
            unit="Bohr")
rhf = mol.RHF.run()
cisolver = pyscf.fci.FCI(rhf)
cisolver.nroots = 5
e0, e1, e2, e3, e4 = cisolver.kernel()[0]
Delta = e1-e0
p = cisolver.ci[0][0,0]**2

converged SCF energy = -74.9629282604051


Generate Hamiltonian

In [4]:
from local_hamiltonian import get_localized_hamiltonian, get_localized_integrals
basis = 'sto3g'
method = 'lowdin'
mol = gto.M(atom="TIP4P-2.xyz",
            basis=basis)
rhf = mol.RHF.run()
e_rhf = rhf.e_tot
of_hamiltonian = get_localized_hamiltonian(mol, method)
constant, one_body_int, two_body_int = get_localized_integrals(mol, method)

converged SCF energy = -149.93597589872
converged SCF energy = -149.93597589872
converged SCF energy = -149.93597589872


Split the Hamiltonian

In [5]:
group1 = list(range(14))
group2 = list(range(14,28))
original_qubit_hamiltonian = openfermion.transforms.jordan_wigner(of_hamiltonian)
constant_in_original_qubit_hamiltonian = original_qubit_hamiltonian.terms[()]
original_qubit_hamiltonian -= constant_in_original_qubit_hamiltonian

In [6]:
from utils import local_group_qubit_operator
hamiltonian_group_1 = local_group_qubit_operator(original_qubit_hamiltonian, group1)
hamiltonian_group_2 = local_group_qubit_operator(original_qubit_hamiltonian, group2)
hamiltonian_interaction = original_qubit_hamiltonian - hamiltonian_group_1 - hamiltonian_group_2

Get the pauli coefficients

In [7]:
def coef2array(h):
    _array = np.array([h.terms[key] for key in h.terms])
    return _array[np.where(np.abs(_array)>1e-11)]
original_coef = coef2array(original_qubit_hamiltonian)
grouped_coef_1 = coef2array(hamiltonian_group_1)
grouped_coef_2 = coef2array(hamiltonian_group_2)
interaction_coef = coef2array(hamiltonian_interaction)

Calculate norms

In [8]:
h1norm = np.sum(np.abs(grouped_coef_1))+np.sum(np.abs(grouped_coef_2)) + abs(e_rhf-constant_in_original_qubit_hamiltonian)
v1norm = np.sum(np.abs(interaction_coef))
v23norm = np.sum(np.abs(interaction_coef)**(2/3))**(3/2)
lvlh = len(interaction_coef)/(len(grouped_coef_1)+len(grouped_coef_2))
print(h1norm)
print(v1norm)
print(v23norm)
print(lvlh)

204.22413253998366
14.381313338906992
782.7464528407681
7.184374075170169


# Generate values in the paper

In [9]:
from cost_estimate import get_cost_for_first_order_pert, get_cost_for_second_order_pert
delta0 = 0.3e-3
delta1 = 0.3e-3
delta2 = 0.3e-3
r_scale = 1/2
epsilon_scale = 1/2
n_subsystems = 2
print(get_cost_for_first_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))
print(get_cost_for_second_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))

epsilon:  3.1683010212839733e-06
n_filter:  (73664.73161987965+0j)
kappa:  0.0019497080245572923
M1:  11900.649959749522
r:  5.215100890479599e-07
x_th: 1.4689742895162745e-06
(53874782773.27172+0j)
r:  1.4450001548320818e-08
epsilon_filter:  8.778728470368974e-08
epsilon_ptb:  2.9623148605915528e-05
kappa:  0.0019497080245572923
w:  0.0019497080245572923
w0:  1.4689742895162745e-06
M2:  155963829859.9959
n_filter:  (93359.45661733639+0j)
n_ptb:  (458388.77517191024+0j)
(1.1779868691018788e+18+0j)
